In [ ]:
import pandas as pd
import csv
import numpy as np
from datetime import datetime

In [ ]:
df3 = pd.read_csv("/Users/asteriag/Desktop/DATASCIENCEPROJECT_WORKFLOW/TimeStamp_Engineering/engineering_AF_segment3.csv",engine='python')

In [ ]:
df3.head(n=10)


In [ ]:
for i in range(len(df3)):
    id=df3.loc[i,'CASE_ID']
    status = df3.loc[i,"WRKFLW_TSK_MAP_GROUPED"]

In [ ]:
ids = list(set(df3['CASE_ID'].values.tolist()))
ids

In [ ]:
S3S1 = pd.DataFrame()

#traverse each id
for id in ids:
    flag = True
    #.loc[::-1].reset_index(drop = True) Reverse row order and reset index which let the first status stay at the first row:
    id_df = df3[df3['CASE_ID'] == id].loc[::-1].reset_index(drop = True)
    #id_df= id_df.iloc[::-1]
    sub_index,sendForAssessment_index,assessmentOutcomeNotS_index, assessmentOutcomeS_index, releaseOffer_index,acceptOrDefer_index = -1,-1,-1,-1,-1,-1
    for index, row in id_df.iterrows():
        #record the index of first 'Submit' status per person
        if row['WRKFLW_TSK_MAP_GROUPED'] == 'Submit'and sub_index == -1:
            sub_index = index
        #record the index of first 'Send for Assessment' status per person
        elif row['WRKFLW_TSK_MAP_GROUPED'] == 'Send for Assessment'and sendForAssessment_index == -1:
            sendForAssessment_index = index
        #record the index of first 'Assessment Outcome S' status per person
        elif (row['WRKFLW_TSK_MAP_GROUPED'] == 'Assessment Outcome - Successful' and assessmentOutcomeS_index == -1):
            assessmentOutcomeS_index = index
        elif (row['WRKFLW_TSK_MAP_GROUPED'] == 'Assessment Outcome - Not Successful'and assessmentOutcomeNotS_index == -1):
            assessmentOutcomeNotS_index = index
        #record the index of first 'Release Offer' status per person
        elif row['WRKFLW_TSK_MAP_GROUPED'] == 'Release Offer' and releaseOffer_index == -1:
            releaseOffer_index = index
        #record the index of first 'Accept/Defer' status per person
        elif (row['WRKFLW_TSK_MAP_GROUPED'] == 'Accept' and acceptOrDefer_index == -1) or\
                (row['WRKFLW_TSK_MAP_GROUPED'] == 'Defer' and acceptOrDefer_index == -1):
            acceptOrDefer_index = index
    #save the person status between submit and sendforassessment into S1
    if sub_index != -1 and sendForAssessment_index != -1 and assessmentOutcomeNotS_index == -1 and assessmentOutcomeS_index == -1\
        and releaseOffer_index == -1 and acceptOrDefer_index == -1:
        df_seg1 = id_df[sub_index:sendForAssessment_index+1]
        S3S1 = pd.concat([S3S1,df_seg1])
        flag = False
    #save the person status from submit and before sendforassessment into S1
    elif sub_index != -1 and sendForAssessment_index == -1 and assessmentOutcomeS_index == -1 and assessmentOutcomeNotS_index == -1\
        and releaseOffer_index == -1 and acceptOrDefer_index == -1:
        df_seg1 = id_df
        S3S1 = pd.concat([S3S1,df_seg1])
        flag = False

